In [1]:
from sqlalchemy import create_engine
from config import Config
from pandas import read_sql
from utils import BinaryEncoder
import datetime

In [6]:
eng = create_engine(Config.DATABASE_URI)
car_heat = read_sql(sql='select * from car_heat', con=eng)
car_heat['time'] = [x.strftime("%H:%M") for x in car_heat['time']]
encoder = BinaryEncoder()
#cat_cols = ['a_rider', 'b_rider', 'c_rider', 'd_rider', 'stadium', 'round', 'year', 'time', 'name_team_home', 'name_team_away']
#encoder.fit(data=car_heat, columns=cat_cols)

In [3]:
encoder.load('test.txt')

In [4]:
encoder.transform(data=car_heat)

In [3]:
encoder.save('test.txt')

In [11]:
encoder.col_cats['time'].strftime("%H:%M")

AttributeError: 'list' object has no attribute 'strftime'

In [12]:
encoder.col_cats['time'] = [x.strftime("%H:%M") for x in encoder.col_cats['time']]

In [4]:
type(encoder.col_cats['time'][0]) == datetime.time

True

In [7]:
car_heat

,heat_id,match_hash,heat_number,a_rider,a_score,a_previous_points,a_previous_heats_no,b_rider,b_score,b_previous_points,...,d_score,d_previous_points,d_previous_heats_no,stadium,round,date,year,time,name_team_home,name_team_away
0,17097,d6864bb66eaa43e705792f455eab6cfe4392e7a445ce0c...,1,Marcin Jędrzejewski,1,0,0,Grzegorz Zengota,0,0,...,2,0,0,Swiss Krono Arena,1,2007-04-08,2007,16:00,ZKŻ Kronopol Zielona Góra,Polonia Bydgoszcz
1,17099,d6864bb66eaa43e705792f455eab6cfe4392e7a445ce0c...,2,Niels Kristian Iversen,1,0,0,Mariusz Staszewski,0,0,...,2,3,1,Swiss Krono Arena,1,2007-04-08,2007,16:00,ZKŻ Kronopol Zielona Góra,Polonia Bydgoszcz
2,17100,d6864bb66eaa43e705792f455eab6cfe4392e7a445ce0c...,3,Grzegorz Walasek,2,0,0,Rafał Okoniewski,3,0,...,1,0,0,Swiss Krono Arena,1,2007-04-08,2007,16:00,ZKŻ Kronopol Zielona Góra,Polonia Bydgoszcz
3,17102,d6864bb66eaa43e705792f455eab6cfe4392e7a445ce0c...,4,Andreas Jonsson,1,0,0,Kevin Woelbert,0,2,...,3,0,0,Swiss Krono Arena,1,2007-04-08,2007,16:00,ZKŻ Kronopol Zielona Góra,Polonia Bydgoszcz
4,17103,d6864bb66eaa43e705792f455eab6cfe4392e7a445ce0c...,5,Zbigniew Suchecki,2,0,1,Mariusz Staszewski,1,0,...,0,0,0,Swiss Krono Arena,1,2007-04-08,2007,16:00,ZKŻ Kronopol Zielona Góra,Polonia Bydgoszcz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14982,34155,3b5d1ef836db07fc1a6373deaf812a99fa52a56bd45d0f...,11,Bartosz Zmarzlik,0,3,4,Jaimon Lidsey,3,4,...,2,9,3,Stadion im. Alfreda Smoczyka,18,2020-10-11,2020,19:15,FOGO UNIA Leszno,MOJE BERMUDY STAL Gorzów
14983,34156,3b5d1ef836db07fc1a6373deaf812a99fa52a56bd45d0f...,12,Dominik Kubera,2,4,2,Rafał Karczmarz,1,7,...,0,0,1,Stadion im. Alfreda Smoczyka,18,2020-10-11,2020,19:15,FOGO UNIA Leszno,MOJE BERMUDY STAL Gorzów
14984,34157,3b5d1ef836db07fc1a6373deaf812a99fa52a56bd45d0f...,13,Anders Thomsen,2,2,3,Dominik Kubera,0,6,...,1,8,3,Stadion im. Alfreda Smoczyka,18,2020-10-11,2020,19:15,FOGO UNIA Leszno,MOJE BERMUDY STAL Gorzów
14985,34158,3b5d1ef836db07fc1a6373deaf812a99fa52a56bd45d0f...,14,Kacper Pludra,3,0,0,Anders Thomsen,1,4,...,0,0,2,Stadion im. Alfreda Smoczyka,18,2020-10-11,2020,19:15,FOGO UNIA Leszno,MOJE BERMUDY STAL Gorzów


In [17]:
car_heat.name_team_away.append(car_heat.name_team_home).unique().tolist()

['Polonia Bydgoszcz',
 'Marma Polskie Folie Rzeszów',
 'Atlas Wrocław',
 'Unia Leszno SSA',
 'Unibax Toruń',
 'CKM Złomrex Włókniarz Częstochowa',
 'Unia Tarnów ŻSSA',
 'ZKŻ Kronopol Zielona Góra',
 'Caelum Stal Gorzów',
 'CKM Cognor Włókniarz Częstochowa',
 'Lotos Wybrzeże Gdańsk',
 'Unia Leszno',
 'Falubaz Zielona Góra',
 'BETARD Sparta Wrocław',
 'CKM Włókniarz Częstochowa',
 'Tauron Azoty Tarnów',
 'Stelmet Falubaz Zielona Góra',
 'PGE Marma Rzeszów',
 'Dospel CKM Włókniarz Częstochowa',
 'Azoty Tauron Tarnów',
 'Stal Gorzów',
 'Unia Tarnów',
 'FOGO Unia Leszno',
 'składywęgla.pl Polonia Bydgoszcz',
 'Lechma Start Gniezno',
 'Grupa Azoty Unia Tarnów',
 'Kantoronline VIPERPRINT Włókniarz Częstochowa',
 'SPAR Falubaz Zielona Góra',
 'Renault Zdunek Wybrzeże Gdańsk',
 'KS Toruń',
 'MRGARDEN GKM Grudziądz',
 'MONEYmakesMONEY.pl Stal Gorzów',
 'PGE Stal Rzeszów',
 'Get Well Toruń',
 'Ekantor.pl Falubaz Zielona Góra',
 'ROW Rybnik',
 'CASH BROKER Stal Gorzów',
 'Włókniarz Vitroszlif Cros

In [ ]:
cities = ['Bydgoszcz', 'Rzeszów', 'Wrocław', 'Leszno', 'Toruń', 'Częstochowa', 'Tarnów', 'Gorzów', 'Zielona Góra', 'Rybnik', 'Ostrów', 'Lublin', 'Grudziądz', 'Gdańsk', 'Gniezno']

In [26]:
car_heat.loc[car_heat['name_team_home'].str.contains('Toruń'), 'name_team_home'] = 'abbb'